In [1]:
%matplotlib inline

In [2]:
# Ensure python 3 forward compatibility
# 保证python3前向兼容
from __future__ import print_function

import numpy as np
import matplotlib.pyplot as plt
import theano
# By convention, the tensor submodule is load as T
# 为了方便使用，张量子模型被加载为T
import theano.tensor as T

# 基础
## Symbolic variables符号变量
在Theano中，所有的算法都是符号化定义的。这种方法更类似书写数学表达式。下列的Theano变量都是符号化的；它们都没有明确的值。

In [3]:
# theano.tensor子模块有很多自带的符号化变量类型
# 在这里，我们定义一个标量（scalar）（0-d）变量
# 在参数中给出变量的名字
foo = T.scalar('foo')
# 现在，我们可以定义另一个变量bar，它的值是foo的平方
bar = foo**2
# 这也是一个theano变量
print(type(bar))
print(bar.type)
# 使用theano的pp（pretty print）函数，我们可以看到
# bar是由foo的平方符号化定义的
print(theano.pp(bar))

<class 'theano.tensor.var.TensorVariable'>
TensorType(float64, scalar)
(foo ** TensorConstant{2})


## 函数
为了使用Theano进行实际的计算，你需要定义符号函数，它可以使用实际值调用并返回一共真实值。

In [4]:
# 我们还不能使用foo和bar进行计算
# 我们需要先定义theano函数
# theano.function中的第一个参数定义了函数的输入
# 需要注意bar依赖于foo，所以foo是这个函数的一共输入
# 在给定foo的值后，theano.function会编译代码来计算bar的值
f = theano.function([foo], bar)
print(f(3))

9.0


In [5]:
# 替代性的，在一些情况下你可以使用一个符号变量的估值方法
# 这种方法比定义一个函数更方便
# 这个估值方法接受一个字典，其中键是theano的变量，值是这些变量的值
print(bar.eval({foo: 3}))

9.0


In [6]:
# 我们也可以使用Python函数来构建Theano变量
# 这看起来很麻烦，但是可以让复杂例子的语法更清晰
def square(x):
    return x**2
bar = square(foo)
print(bar.eval({foo: 3}))

9.0


## theano.tensor
Theano也有向量、矩阵、张量的变量类型。theano.tensor子模块有很多函数可以操作这些变量。

In [7]:
A = T.matrix('A')
x = T.vector('x')
b = T.vector('b')
y = T.dot(A, x) + b
# 注意对一个矩阵做平方操作是element-wise
z = T.sum(A**2)
# theano.function可以同时进行很多算
# 你也可以设定参数的默认值
# 我们会在后续涉及到theano.config.floatX
b_default = np.array([0, 0], dtype=theano.config.floatX)
linear_mix = theano.function([A, x, theano.Param(b, default=b_default)], [y, z])
# 为A, x, b设定值
print(linear_mix(np.array([[1, 2, 3],
                           [4, 5, 6]], dtype=theano.config.floatX), #A
                 np.array([1, 2, 3], dtype=theano.config.floatX), #x
                 np.array([4, 5], dtype=theano.config.floatX))) #b
# 使用b的默认值
print(linear_mix(np.array([[1, 2, 3],
                           [4, 5, 6]], dtype=theano.config.floatX), #A
                 np.array([1, 2, 3], dtype=theano.config.floatX))) #x

[array([ 18.,  37.]), array(91.0)]
[array([ 14.,  32.]), array(91.0)]


## Shared variables共享变量
共享变量有一些不同，它们实际上有一个明确的值，并且可以被获取和设置，同时在跨函数使用时共享这些值。它们是很方便的，因为它们在跨函数调用的时候有声明。

In [8]:
shared_var = theano.shared(np.array([[1, 2], [3, 4]], dtype=theano.config.floatX))
# 共享变量的数据类型可以由它的初始化过程推导出
print(shared_var.type())

<TensorType(float64, matrix)>


In [9]:
# 我们可以使用set_value设置一个共享变量的值
shared_var.set_value(np.array([[3, 4], [2, 1]], dtype=theano.config.floatX))
# 并且使用get_value获取值
print(shared_var.get_value())

[[ 3.  4.]
 [ 2.  1.]]


In [10]:
shared_squared = shared_var**2
# theano.function的第一个参数（inputs）告诉Theano编译函数的参数应该是怎么样的
# 由于shared_var是共享的，它已经有了值，所以它没有必要作为一个函数的输入
# 因此，当使用shared_squared时，Theano隐式地将shared_var作为一个函数的输入
# 我们不需要在theano.function的输入变量中包含它
function_1 = theano.function([], shared_squared)
print(function_1())

[[  9.  16.]
 [  4.   1.]]


## updates 更新
共享变量的值可以通过使用theano.function的升级参量来更新

In [11]:
# 我们也可以通过一个函数更新共享变量的声明
substract = T.matrix('substract')
# 更新需要使用一共字典，其中的键为共享变量，值为共享变量的新值
# 这里，更新会设置shared_var = shared_var - substract
function_2 = theano.function([substract], shared_var, updates={shared_var: shared_var - substract})
print("shared_var before substracting [[1, 1], [1, 1]] using function_2:")
print(shared_var.get_value())
# 从shared_var中减去[[1, 1], [1, 1]]
function_2(np.array([[1, 1], [1, 1]], dtype=theano.config.floatX))
print("shared_var after calling function_2:")
print(shared_var.get_value())
# 注意这种方式也改变function_1的输出，应为shared_var是共享的
print("New output of function_1() (shared_var*2):")
print(function_1())

shared_var before substracting [[1, 1], [1, 1]] using function_2:
[[ 3.  4.]
 [ 2.  1.]]
shared_var after calling function_2:
[[ 2.  3.]
 [ 1.  0.]]
New output of function_1() (shared_var*2):
[[ 4.  9.]
 [ 1.  0.]]


## Gradients 梯度
使用Theano的一个很大的好处是它可以计算梯度。这可以让你符号化地定义一个函数并且快速计算它的（数学上的）导数，而不用真正去求导。

In [12]:
# 回想bar = foo**2
# 我们可以计算bar的梯度，而不用考虑foo：
bar_grad = T.grad(bar, foo)
# 我们期望的是bar_grad = 2**foo
bar_grad.eval({foo: 10})

array(20.0)

In [13]:
# 回想 y = Ax + b
# 我们可以用如下方法计算雅可比矩阵：
y_J = theano.gradient.jacobian(y, x)
linear_mix_J = theano.function([A, x, b], y_J)
# 因为这是一个线性混合，我们期望的输出永远是A
print(linear_mix_J(np.array([[9, 8, 7], [4, 5, 6]], dtype=theano.config.floatX), #A
                   np.array([1, 2, 3], dtype=theano.config.floatX), #x
                   np.array([4, 5], dtype=theano.config.floatX))) #b
# 我们也可以使用theano.gradient.hessian来计算黑塞矩阵Hessian

[[ 9.  8.  7.]
 [ 4.  5.  6.]]


## Debugging 调试
在Theano上调试会有一些困难，因为代码实际运行的时候，和你写的代码已经相去甚远。一种在实际编译函数之前，简便检查你的Theano表达式的方法就是使用测试变量。

In [14]:
# 让我们创建另一个矩阵“B”
B = T.matrix('B')
# 和一个符号变量表示A和B的点乘
# 在这种情况，Theano并不知道A和B的形状shapes，所以没有办法知道A点乘B是否是有效的。
C = T.dot(A, B)
# 现在让我们使用它
C.eval({A: np.zeros((3, 4), dtype=theano.config.floatX),
        B: np.zeros((5, 6), dtype=theano.config.floatX)})

ValueError: Shape mismatch: x has 4 cols (and 3 rows) but y has 5 rows (and 6 cols)
Apply node that caused the error: Dot22(A, B)
Inputs types: [TensorType(float64, matrix), TensorType(float64, matrix)]
Inputs shapes: [(3, 4), (5, 6)]
Inputs strides: [(32, 8), (48, 8)]
Inputs values: ['not shown', 'not shown']

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.
HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.

上面的错误信息有一些不透明（并且如果我们没有给定Theano变量A和B的名字的时候会更难阅读）。当Theano表达式很复杂的时候，会变得尤为难懂。它也不会告诉你在Python代码中出现错误的行号，因为实际运行的并不是你的Python代码，而是编译后的Theano代码！还好，“test values”可以让我们处理这个问题。但是，并不是所有Theano方法都允许测试值（test values)（例如 scan方法）

In [16]:
# 这告诉Theano我们将要使用test values，并且当出现错误时发出警告。
# 设置‘warn’表示“当我没有提供一个测试值时警告我”
theano.config.compute_test_value = 'warn'
# 设置参数tag.test_value，赋予变量它的测试值
A.tag.test_value = np.random.random((3, 4)).astype(theano.config.floatX)
B.tag.test_value = np.random.random((5, 6)).astype(theano.config.floatX)
# 现在，当我们计算C的时候会得到错误信息，指出我们需要修正的行
C = T.dot(A, B)

ValueError: shapes (3,4) and (5,6) not aligned: 4 (dim 1) != 5 (dim 0)

In [17]:
# 我们在接下来的教程中将不使用测试值
theano.config.compute_test_value = 'off'

使用调试的另一个有用的地方是，当一个无效的计算被执行的时候，例如结果是nan。默认情况下，Theano会静默地允许这些nan值被使用和计算，但是这种默认操作会导致你的Theano计算产生灾难性的后果。如果时间允许，我们可以选择在DebugMode使用Theano编译函数，由于一个无效计算会产生一个错误。

In [18]:
# 一个简单的除法函数
num = T.scalar('num')
den = T.scalar('den')
divide = theano.function([num, den], num/den)
print(divide(10, 2))
# 这会导致一个NaN
print(divide(0, 0))

5.0
nan


In [19]:
# 要在编译模式下编译一个方程，只需要设置mode='DebugMode'
divide = theano.function([num, den], num/den, mode='DebugMode')
# NaNs现在会产生一个错误
print(divide(0, 0))

InvalidValueError: InvalidValueError
        type(variable) = TensorType(float64, scalar)
        variable       = Elemwise{true_div,no_inplace}.0
        type(value)    = <type 'numpy.ndarray'>
        dtype(value)   = float64
        shape(value)   = ()
        value          = nan
        min(value)     = nan
        max(value)     = nan
        isfinite       = False
        client_node    = None
        hint           = perform output
        specific_hint  = non-finite elements not allowed
        context        = ...
  Elemwise{true_div,no_inplace} [@A] ''   
   |num [@B]
   |den [@C]

        